# Coles Online Interstate Differences - Exploratory Analysis

## Purpose

The objective of this exploratory analysis is to ascertain whether Coles charges different prices for the same product in different states. This would imply that Coles passes on supply chain costs to customers who are in areas further away from their main distribution centres. It may also give us a very general idea of where the main distribution centres are located. Insights gained from this analysis may provide a more granular picture of Coles' pricing strategy - which may be useful in formulating pricing and/or market segmentation strategies to compete with Coles.

If price differences are not found between major cities and remote regions of Australia, that would imply that customers in major city centres are _de facto_ subsidising remote area customers' online purchases from Coles.

Date data extracted: _26 January 2021_ 

Coles Online Geolocations: 
* Adelaide SA 
* Melbourne VIC

### Loading data into Pandas and preliminary data scrubbing 


Raw data scraped from Coles often has many data integrity issues. The most common ones being:
* Unavailable Products are listed
* Products listed as available have no price tags (i.e. it's actually unavailable but their system is either bugged or someone didn't make the correct changes on their internal system)
* Duplicate products (i.e. same SKU from links with promotional suffix in the URL which escaped my scraper's filtering)

Below, basic data scrubbing was applied while reading in the CSV files into Pandas.

* Replaced 'NaN' values with an empty string.
* Removed duplicate items with identical SKUs
* Date column removed - only useful in timeseries analysis
* Applied filter to only load in products with the 'Available' tag and a 'Current Price' value

In [49]:
import pandas as pd

df_adl = pd.read_csv('sauce/Jan 26 Adelaide SA.csv').fillna('').drop_duplicates(subset=['SKU']).drop(columns=['Date'])
df_adl = df_adl.loc[(df_adl['Availability'] == 'Available') & (df_adl['Current Price'] != '')]

df_mel = pd.read_csv('sauce/Jan 26 Melbourne VIC.csv').fillna('').drop_duplicates(subset=['SKU']).drop(columns=['Date'])
df_mel = df_mel.loc[(df_mel['Availability'] == 'Available') & (df_mel['Current Price'] != '')]

Let's double check that Coles indeed uses the same SKU numbers for the same products across Australia.

To do this, we'll sort the SKU column and eyeball the first few items of both dataframes.

In [50]:
# Adelaide
df_adl.sort_values(by='SKU').head()

,SKU,Brand,Product Name,Current Price,Price per unit,Unit Size,Previous Price,On Special,Suppliers,Category,Availability,URL
8897,1000301P,Real Foods,Tasty Cheese Corn Thins,1.80,$1.44 per 100G,125g,2.0,$pecials,['731170 REAL FOODS'],Summer Entertaining,Available,https://shop.coles.com.au/a/churchill-centre/product/real-foods-corn-thins-tasty-cheese
20151,10016P,Victoria Bitter,Can 375mL,58.00,$6.44 per 1L,24 Pack,,,['9393 CUB P/L'],Liquor,Available,https://shop.coles.com.au/a/churchill-centre/product/victoria-bitter-375ml-cans-10016p
19012,100537P,Steradent,Powder Denture Cleansing,5.95,$2.98 per 100G,200g,,,['672882 RECKITT BENCKISER (AUST) P/L'],Health & Beauty,Available,https://shop.coles.com.au/a/churchill-centre/product/steradent-denture-cleansing-powder
7076,1005430P,Coles Finest,Angus Beef Eye Fillet Steak,16.25,$65.00 per 1Kg,approx. 250g pack,,,"['963906 COLES MT PROD - C9687MT - MAP WA', '964437 MAP WA P/L']",Meat & Seafood,Available,https://shop.coles.com.au/a/churchill-centre/product/coles-finest-steak-angus-beef-eye-fillet
7404,1005611P,Coles Finest,Angus Beef Scotch Fillet Steak,17.55,$50.00 per 1Kg,approx. 351g each,,,"['964437 MAP WA P/L', '963906 COLES MT PROD - C9687MT - MAP WA']",Meat & Seafood,Available,https://shop.coles.com.au/a/churchill-centre/product/coles-finest-steak-angus-beef-scotch-fillet


In [51]:
# Melbourne
df_mel.sort_values(by='SKU').head()

,SKU,Brand,Product Name,Current Price,Price per unit,Unit Size,Previous Price,On Special,Suppliers,Category,Availability,URL
10948,1000301P,Real Foods,Tasty Cheese Corn Thins,1.80,$1.44 per 100G,125g,2.0,$pecials,['731170 REAL FOODS'],Summer Entertaining,Available,https://shop.coles.com.au/a/richmond-south/product/real-foods-corn-thins-tasty-cheese
685,10016P,Victoria Bitter,Can 375mL,48.00,$5.33 per 1L,24 Pack,53.0,$pecials,['9393 CUB P/L'],Liquor,Available,https://shop.coles.com.au/a/richmond-south/product/victoria-bitter-375ml-cans-10016p
8915,10030P,Melbourne Bitter,Can 375mL,58.00,$6.44 per 1L,24 Pack,,,['9393 CUB P/L'],Liquor,Available,https://shop.coles.com.au/a/richmond-south/product/melbourne-bitter-can-375ml
7052,100537P,Steradent,Powder Denture Cleansing,5.95,$2.98 per 100G,200g,,,['672882 RECKITT BENCKISER (AUST) P/L'],Health & Beauty,Available,https://shop.coles.com.au/a/richmond-south/product/steradent-denture-cleansing-powder
659,1007071P,Babylove,Unisex Stretchy Walker Nappy Pants 12-17 Kg Size 5,11.80,$0.47 per 1Ea,25 pack,17.0,$pecials,['386332 UNICHARM AUSTRALASIA'],Baby,Available,https://shop.coles.com.au/a/richmond-south/product/babylove-nappy-pants-walker


Confirmed that SKUs indeed reference the exact same product.

Interestingly enough, we can already see that some products have a pretty significant price difference *e.g. A slab of Victoria Bitter has a \\$10 difference*. Other products seem to be priced the same.


One thing I noticed from reviewing other data, the "National" data (which can be obtained from the list of product URLs located on their sitemap) doesn't have any actual alcoholic products listed under the 'Liquor' category. Judging from this peculiarity, perhaps, the reason might be because those are the only products that differ in price across Australia??

It's also worth noting Coles technically doesn't sell alcohol (I've never seen alcohol stocked in any Coles physical stores). Perhaps these prices are from Liquorland?

## Product Availability Differences

* How many products are available on 26 January 2021, in Coles' online shop for Adelaide and Melbourne?
* How many products do they have in common?
* How many products are exclusive at each store on this particular day?

Finally: Output lists of Common Products and Exclusive Products of each location as CSV files.

#### How many products are available in Adelaide and Melbourne, respectively?

In [52]:
print(f'Adelaide has {len(df_adl)} available products')
print(f'Melbourne has {len(df_mel)} available products')

Adelaide has 19297 available products
Melbourne has 14409 available products


#### How many products in common?

In [53]:
# Shorten original dataframes to only have the necessary columns

adl_df = df_adl[['SKU', 'Brand', 'Product Name', 'Category', 'On Special', 'Current Price', 'Unit Size', 'Suppliers']]
adl_df = adl_df.rename(columns={
    'On Special': 'ADL Specials',
    'Current Price': 'ADL Price',
    'Unit Size': 'ADL packaging',
    'Suppliers': 'ADL Suppliers'
})

mel_df = df_mel[['SKU', 'Brand', 'Product Name', 'Category', 'On Special', 'Current Price', 'Unit Size', 'Suppliers']]
mel_df = mel_df.rename(columns={
    'On Special': 'MEL Specials',
    'Current Price': 'MEL Price',
    'Unit Size': 'MEL packaging',
    'Suppliers': 'MEL Suppliers'
})

In [54]:
# Inner Join of both dataframes and re-arrangements to juxtapose relevant columns

adl_mel_df = pd.merge(adl_df, mel_df, on=['SKU', 'Brand', 'Product Name', 'Category'], how='inner')

# Rename columns to reflect location attribute

adl_mel_df = adl_mel_df[['SKU', 'Brand', 'Product Name', 'Category',
                         'ADL Price', 'MEL Price',
                         'ADL packaging', 'MEL packaging',
                         'ADL Specials', 'MEL Specials',
                         'ADL Suppliers', 'MEL Suppliers']]

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 200)
adl_mel_df.head(100)

,SKU,Brand,Product Name,Category,ADL Price,MEL Price,ADL packaging,MEL packaging,ADL Specials,MEL Specials,ADL Suppliers,MEL Suppliers
0,3468031P,Coles,Natures Kitchen Lentil Spag Bolognese,"Dairy, Eggs & Meals",7.00,7.00,350g,350g,$pecials,$pecials,"['958543 JEWEL OF INDIA ENTERPRISES P/L (', '979047 CHEF FRESH PTY LTD (JFF) (CB)']","['958543 JEWEL OF INDIA ENTERPRISES P/L (', '979047 CHEF FRESH PTY LTD (JFF) (CB)']"
1,2649234P,Guylian,Assortment Belgian Chocolates,Pantry,18.00,18.00,305g,305g,,,['945407 STUART ALEXANDER (CONFECTIONERY)'],['945407 STUART ALEXANDER (CONFECTIONERY)']
2,8416240P,Heinz,Big Red Tomato Soup Can,Pantry,3.65,3.65,820g,820g,,,['22161 HJ HEINZ CO AUST LTD'],['22161 HJ HEINZ CO AUST LTD']
3,2592829P,Jack Daniels,Double Jack & Cola Can 375mL,Liquor,62.00,59.00,10 pack,10 pack,,,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']
4,324217P,Coles,Smart Buy Hair Spray,Health & Beauty,2.20,2.20,250g,250g,,,"['655091 AARON LABO', '632775 MULTI-FILL P/L HOUSEBRAND']","['655091 AARON LABO', '632775 MULTI-FILL P/L HOUSEBRAND']"
5,5143435P,Nivea,Moisturising Day Face Cream Fluid Spf15,Health & Beauty,5.35,5.35,50mL,50mL,$pecials,$pecials,['936507 BEIERSDORF AUST LTD'],['936507 BEIERSDORF AUST LTD']
6,3716508P,Peters,Connoisseur Strawberry Ice Cream,Frozen,11.00,11.00,1L,1L,,,['968585 PETERS ICE CREAM'],['968585 PETERS ICE CREAM']
7,5430233P,Earth Choice,Pure Soft Fabric Softener,Household,2.00,2.00,1L,1L,,,['653217 NATURES ORGANICS P/L'],['653217 NATURES ORGANICS P/L']
8,3116216P,Cook & Dine,Corkscrew,Household,3.00,3.00,1 each,1 each,,,"['945707 GLOBALEYES DISTRIBUTION P/L (XDO', '970149 Globaleyes Distribution (COLESBR', '975760 GLOBALEYES DISTRIBUTION PTY LTD']","['945707 GLOBALEYES DISTRIBUTION P/L (XDO', '970149 Globaleyes Distribution (COLESBR', '975760 GLOBALEYES DISTRIBUTION PTY LTD']"
9,3671057P,Grillman,BBQ Aluminium Large Roasting Trays,Household,4.80,4.80,2 pack,2 pack,$pecials,$pecials,['972884 MAYO HARDWARE PTY LTD'],['972884 MAYO HARDWARE PTY LTD']


In [55]:
print(f'There are {len(adl_mel_df)} items in common between Adelaide and Melbourne')

There are 12523 items in common between Adelaide and Melbourne


#### How many products are available in Adelaide but not in Melbourne?

In [100]:
# Anti merge by using the indicator tag to get the products exclusive to Adelaide (left dataframe)

df_outter_join = pd.merge(adl_df, mel_df, on=['SKU', 'Brand', 'Product Name', 'Category'], how="outer", indicator=True)

unique_adl_df = df_outter_join[['SKU', 'Brand', 'Product Name', 
                                'Category', 'ADL Price', 'ADL Suppliers']].loc[df_outter_join['_merge'] == 'left_only']
unique_adl_df.head(20)

,SKU,Brand,Product Name,Category,ADL Price,ADL Suppliers
0,3833501P,Farmers Market,Homestyle Shreds With Chicken Carrot & Cranberries Dog Food,Pet,13.0,['721182 VIP PETFOODS (AUST) P/L (CHILLED']
5,2015360P,1000 Hour,Black Eyelash & Brow Dye Kit,Health & Beauty,20.0,"['145823 OVERSEAS SUPPLIER', '942368 CHEMCORP INTL', '969600 Chemcorp pty Ltd']"
9,3710829P,Tresemme,Violet Blonde Shine Conditioner,Health & Beauty,13.0,['26867 UNILEVER A/ASIA HOME &PERSONAL C']
10,3614596P,Tigi,Bed Head reatment Mask urban Antidotes L2 Recovery,Health & Beauty,20.9,['26867 UNILEVER A/ASIA HOME &PERSONAL C']
13,3830977P,Coles,Seashore Royale Platter,From The Deli,55.0,['656186 COLES SUPERMARKETS (EXP)']
18,3339390P,Daawat,Everyday Basmati Rice Gold,International Foods,14.0,"['970023 Sabi Foods', '972534 SABRINI FOODS P/L']"
21,9125050P,Kirks,Lemon Squash Multipack Cans 375 mL,Summer Entertaining,6.5,"['931858 COCA COLA AMATIL AUST P/L (DIREC', '931859 COCA COLA AMATIL AUST P/L (GHPL)', '37079 LACTALIS AUSTRALIA P/L']"
24,2709722P,Balfours Premium,Peppersteak Pie,"Dairy, Eggs & Meals",4.2,['954304 BALFOURS BAKERY P/L']
25,3836328P,Olay,Hydrating Caviar Lime Scrubs,Health & Beauty,19.0,['422720 PROCTER & GAMBLE AUST P/L']
29,3761502P,Kynd,Glow Hair Skin & Nails,Health & Beauty,30.0,['INOVA PHARMACEUTICALS PTY LTD']


In [57]:
print(f"There are {len(unique_adl_df)} only available in Adelaide which are not available in Melbourne")

There are 6774 only available in Adelaide which are not available in Melbourne


#### Category breakdown of those products.

In [58]:
unique_adl_df['Category'].value_counts()

Health & Beauty         1799
Household                876
Pantry                   744
Dairy, Eggs & Meals      457
Pet                      388
Frozen                   371
Back to School           367
International Foods      310
Bakery                   296
Meat & Seafood           255
Summer Entertaining      197
Drinks                   186
Fruit & Vegetables       138
Liquor                   132
Baby                     111
From The Deli             64
                          31
Convenience Meals         28
Clothing                  22
New                        1
Chilled & Fresh Food       1
Name: Category, dtype: int64

#### How many products are available in Melbourne but not in Adelaide?

In [101]:
unique_mel_df = df_outter_join[['SKU', 'Brand', 'Product Name', 
                                'Category', 'MEL Price', 'MEL Suppliers']].loc[df_outter_join['_merge'] == 'right_only']
unique_mel_df.head(20)

,SKU,Brand,Product Name,Category,MEL Price,MEL Suppliers
19297,3387381P,Deeko,Collection Dinner Plate,Summer Entertaining,5.00,['4545 THE SORBENT PAPER CO']
19298,3312600P,Multix,ReuseMe Resealable Snack Bags,Back to School,2.40,"[""8125 MCPHERSON'S CONSUMER PRODUCTS P/""]"
19299,3292886P,Femfresh,Pure & gentle For Sensitive Skin Intimate Care,Health & Beauty,6.75,['7641 CHURCH & DWIGHT (AUST) P/L']
19300,11549P,Budweiser,Bottle 330mL,Liquor,19.00,"['169871 CASTLEMAINE PERKINS LTD', '42196 THE SWAN BREWERY CO P/L', '975969 LION-BEER SPIRITS & WINE PTY LTD', '729248 AUSTRALIAN LIQUOR MARKETERS P/L', '710911 LION NATHAN VICTORIA P/L (BEER)', '..."
19301,6100573P,Saint Cosme,Cotes du Rhone 750mL,Liquor,22.00,"['12947 G.H.P.L.', '963979 MAX LIQUOR SUPPLIES']"
19302,3519711P,KOi,Arabica Coffee & Shea Butter Body Scrub,Health & Beauty,5.60,"['976000 TILLEY SOAPS AUSTRALIA P/L (CB)', '939013 COLES SUPERMARKETS P/L']"
19303,9759886P,Pauls,Smarter White Milk,"Dairy, Eggs & Meals",5.55,"['944347 LACTALIS AUSTRALIA P/L', '37079 LACTALIS AUSTRALIA P/L', '964585 LACTALIS AUSTRALIA P/L', '964518 PARMALAT FOOD PRODUCTS P/L (NSW', '959594 PARMALAT FOOD PRODUCTS P/L']"
19304,3911809P,Primo,Shaved Champange Ham,Back to School,7.00,['959437 PRIMO FOODS PTY LTD']
19305,3766459P,Rafferty's Garden,Summer Fruits & Quinoa 6M+,Baby,1.90,"[""970237 RAFFERTY'S GARDEN AUST""]"
19306,3114050P,Land of Giants,Pinot Gris 750mL,Liquor,17.00,['963979 MAX LIQUOR SUPPLIES']


In [60]:
print(f"There are {len(unique_mel_df)} products unique to Melbourne")

There are 1886 products unique to Melbourne


#### Category Breakdown

In [61]:
unique_mel_df['Category'].value_counts()

Liquor                 509
Pantry                 357
Health & Beauty        195
Dairy, Eggs & Meals    114
Meat & Seafood          87
Summer Entertaining     85
Back to School          85
Household               83
Baby                    77
Drinks                  65
International Foods     60
Frozen                  54
Pet                     33
Fruit & Vegetables      32
Bakery                  31
                        13
From The Deli            3
Convenience Meals        3
Name: Category, dtype: int64

#### Outputting DataFrames to CSV

In [62]:
# Output all the above dataframes to CSV for better viewing in Excel

# Common Products List
adl_mel_df.to_csv('Outputs/Common Products ADL-MEL.csv', index=False, encoding='utf-8-sig')

# Unique to Adelaide
unique_adl_df.to_csv('Outputs/Unique to ADL.csv', index=False, encoding='utf-8-sig')

# Unique to Melbourne
unique_mel_df.to_csv('Outputs/Unique to MEL.csv', index=False, encoding='utf-8-sig')

#### Summary Insights:

* Adelaide had __19,297__ available products. Melbourne had __14,409__ available products
* There were __12,523__ items in common
* Adelaide had __6774__ products which were unavailable in the Melbourne online store. The majority of these products were in the _'Health & Beauty', 'Household'_ and _'Pantry'_ categories.
* Melbourne had __1886__ products which were unavailable in the Adelaide online store. The majority of these products were in the _'Liquor', 'Pantry'_ and _'Health & Beauty'_ categories.

<mark>See the three CSV file dumps in the 'Outputs' folder for further details.</mark>

<br>

## Suppliers

* Are all the suppliers the same for all of the products in common?
* Who are the suppliers of the products only available in Adelaide? (and vice versa)
* Are there any products with different suppliers?

#### Are all the suppliers the same for all the common products? (most likely, yes, but i'm curious)

In [63]:
# Out of curiosity... are the suppliers all the same?

filter = adl_mel_df['ADL Suppliers'] != adl_mel_df['MEL Suppliers']

different_suppliers_df = adl_mel_df.loc[filter]

print(f"Products with different suppliers: {len(different_suppliers_df)}")

Products with different suppliers: 0


#### Suppliers of Adelaide unique products not listed as a supplier in the Melbourne store

In [64]:
filter = unique_adl_df['ADL Suppliers'].isin(df_mel['Suppliers'])

unique_adl_df[~filter][['Brand', 'Product Name', 'Category', 'ADL Suppliers']]

,Brand,Product Name,Category,ADL Suppliers
5,1000 Hour,Black Eyelash & Brow Dye Kit,Health & Beauty,"['145823 OVERSEAS SUPPLIER', '942368 CHEMCORP INTL', '969600 Chemcorp pty Ltd']"
18,Daawat,Everyday Basmati Rice Gold,International Foods,"['970023 Sabi Foods', '972534 SABRINI FOODS P/L']"
21,Kirks,Lemon Squash Multipack Cans 375 mL,Summer Entertaining,"['931858 COCA COLA AMATIL AUST P/L (DIREC', '931859 COCA COLA AMATIL AUST P/L (GHPL)', '37079 LACTALIS AUSTRALIA P/L']"
24,Balfours Premium,Peppersteak Pie,"Dairy, Eggs & Meals",['954304 BALFOURS BAKERY P/L']
30,Beijing,Sliced Garlic Tofu Pieces,"Dairy, Eggs & Meals",['935403 BEIJING SOYA BEAN PRODUCTS P/L']
...,...,...,...,...
19208,Coles,Brussels Sprouts loose,Fruit & Vegetables,"['414760 ST GEORGE PROD PTY LTD', '939312 FRESH PRODUCE DC (NSW)', '942633 CARTER & SPENCER NORTH QLD', '97446 CARTER & SPENCER GROUP P/L', ""635235 COSTA'S P/L (VIC DC)"", '933347 COSTAS (TASMANIA)..."
19215,Ashton Valley Crush,Full Bodied Pear Juice,Drinks,"['942529 R CERAVOLO & CO P/L', '983253 CERAVOLO ORCHARDS', '966748 CERAVOLO ORCHARDS PTY LTD']"
19224,Ivans,Apple Crumble,Bakery,['945657 IVANS PIES (PRIVATE LABEL)']
19254,Maybelline,Matte Maker Natural Beige 30 Powder,Health & Beauty,"[""720488 L'OREAL AUST P/L"", ""952096 L'OREAL AUST P/L (X-DOCK)""]"


#### Suppliers of Melbourne unique products not listed as a supplier in the Adelaide store

In [65]:
filter = unique_mel_df['MEL Suppliers'].isin(df_adl['Suppliers'])

unique_mel_df[~filter][['Brand', 'Product Name', 'Category', 'MEL Suppliers']]

,Brand,Product Name,Category,MEL Suppliers
19300,Budweiser,Bottle 330mL,Liquor,"['169871 CASTLEMAINE PERKINS LTD', '42196 THE SWAN BREWERY CO P/L', '975969 LION-BEER SPIRITS & WINE PTY LTD', '729248 AUSTRALIAN LIQUOR MARKETERS P/L', '710911 LION NATHAN VICTORIA P/L (BEER)', '..."
19303,Pauls,Smarter White Milk,"Dairy, Eggs & Meals","['944347 LACTALIS AUSTRALIA P/L', '37079 LACTALIS AUSTRALIA P/L', '964585 LACTALIS AUSTRALIA P/L', '964518 PARMALAT FOOD PRODUCTS P/L (NSW', '959594 PARMALAT FOOD PRODUCTS P/L']"
19308,Coles,Traditional Fruit Hot Cross Buns,Bakery,"['29564 CRIPPS NUBAKE', '652199 TIP TOP', '953879 ALLIED MILLS P/L', '972720 ALLIED PINNACLE', '920925 TIP TOP IN-STORE BAKERY PRODUCTS', '937127 CRIPPS NUBAKE P/L', '965447 DAVIES BAKERY P/L (HOU..."
19310,Tibaldi,Turkey Breast,Back to School,"['974006 TIBALDI AUSTRALASIA(FIXED WEIGHT', '968633 TIBALDI A/ASIA P/L (DY)']"
19323,Breathe Right,Clear Regular Nasal Congestion Strips,Health & Beauty,"['617857 MARKETING BROKERS AUST P/L', '971903 GLAXOSMITHKLINE CONSUMER HEALTHC', '41629 GLAXOSMITHKLINE AUST P/L']"
...,...,...,...,...
21157,Coles,Australian Peri Peri Roast Chicken Breast Slices,"Dairy, Eggs & Meals","['922163 MILNE AGRIGROUP P/L', '970470 THE CORDINA FOOD CO (COLES BRAND']"
21161,Dare,Espresso Iced Coffee,"Dairy, Eggs & Meals","['966403 LD&D AUSTRALIA PTY LTD (DSD)', '367702 CAPITOL CHILLED FOODS (AUST) P/L', '665282 LD & D AUST P/L']"
21163,Caprilac,Pasteurised Goats Milk,"Dairy, Eggs & Meals","['969734 THE INFANT FOOD CO. PTY LIMITED', '973144 NULAC FOODS PTY LTD']"
21174,Aqualove,Alkaline Water,Back to School,"['974947 AQUALOVE WATER', '145823 OVERSEAS SUPPLIER']"


In [66]:
mel_suppliers_df = pd.read_csv('sauce/Jan 26 Melbourne VIC.csv')[
    ['SKU', 'Brand', 'Product Name', 'Suppliers']].rename(columns={'Suppliers': 'MEL Suppliers'})

adl_suppliers_df = pd.read_csv('sauce/Jan 26 Adelaide SA.csv')[
    ['SKU', 'Brand', 'Product Name', 'Suppliers']].rename(columns={'Suppliers': 'ADL Suppliers'})

In [67]:
# Suppliers found in Melbourne but not in Adelaide

filter = mel_suppliers_df['MEL Suppliers'].isin(adl_suppliers_df['ADL Suppliers'])

df_MelbSuppliersNotListedInADL = mel_suppliers_df[~filter]
df_MelbSuppliersNotListedInADL

,SKU,Brand,Product Name,MEL Suppliers
58,9759886P,Pauls,Smarter White Milk,"['944347 LACTALIS AUSTRALIA P/L', '37079 LACTALIS AUSTRALIA P/L', '964585 LACTALIS AUSTRALIA P/L', '964518 PARMALAT FOOD PRODUCTS P/L (NSW', '959594 PARMALAT FOOD PRODUCTS P/L']"
99,3939706P,Breadtop,Pork Floss Mayo Sandwich,['977923 BREADTOP']
128,8371390P,Coles,Traditional Fruit Hot Cross Buns,"['29564 CRIPPS NUBAKE', '652199 TIP TOP', '953879 ALLIED MILLS P/L', '972720 ALLIED PINNACLE', '920925 TIP TOP IN-STORE BAKERY PRODUCTS', '937127 CRIPPS NUBAKE P/L', '965447 DAVIES BAKERY P/L (HOU..."
145,3331831P,Tibaldi,Turkey Breast,"['974006 TIBALDI AUSTRALASIA(FIXED WEIGHT', '968633 TIBALDI A/ASIA P/L (DY)']"
263,3780675P,The Organic Milk Company,Salted Butter,['976052 TOMC BRANDS PTY LTD (CB)']
...,...,...,...,...
15947,3034010P,Dare,Espresso Iced Coffee,"['966403 LD&D AUSTRALIA PTY LTD (DSD)', '367702 CAPITOL CHILLED FOODS (AUST) P/L', '665282 LD & D AUST P/L']"
15974,3987232P,Caprilac,Pasteurised Goats Milk,"['969734 THE INFANT FOOD CO. PTY LIMITED', '973144 NULAC FOODS PTY LTD']"
15999,3112678P,Cook & Dine,Wooden Pastry Brush,"['970149 Globaleyes Distribution (COLESBR', '971613 GROUPE SEB AUSTRALIA PTY LTD', '945707 GLOBALEYES DISTRIBUTION P/L (XDO']"
16044,3250300P,Aqualove,Alkaline Water,"['974947 AQUALOVE WATER', '145823 OVERSEAS SUPPLIER']"


#### Are there any products with different suppliers

In [94]:
'''
Are there any products in the list above (Suppliers not found in Adelaide) that can be found in Adelaide 
i.e. same product, different supplier
'''

filter = adl_suppliers_df['SKU'].isin(df_MelbSuppliersNotListedInADL['SKU'])
adl_suppliers_df[filter]
print(f'Total number of products in Adelaide with different suppliers to Melbourne: {len(adl_suppliers_df[filter])}')

Total number of products in Adelaide with different suppliers to Melbourne: 0


#### Inferences Drawn

Initially, it seemed like there were quite a few number of suppliers supplying one city and not the other. The issue with the initial outputs was the fact that I had filtered out the "unavailable" products right at the start. After opening up the CSV files in Excel and viewing them, it turns out some of the products e.g. Budweiser does have the exact same list of suppliers but was listed as "unavailable" in Adelaide.

So, I reloaded the CSV files into new dataframes, this time including all of the "unavailable" products. The results were pretty much as expected. The suppliers not found in the Melbourne stores were for all of the products which were also "unavailable" or not listed in the Melbourne store. It's highly likely that to be a supplier for Coles, one must have the capability to supply Nationally. However, since Coles lists a list of suppliers, it's possible that some of them only supply exclusively to one area.

<br>

## Melbourne vs Adelaide - Pricing Differences


* What products have the same price? What categories are most prevelant?
* What are the identical products with different prices?
* Are all the same products on "special" between these two cities?
* Are pricing differences due to only one store having that product on "special"?
* Are there any trends in these pricing differences i.e. are the pricing differences heavily within one category or is it influenced by brand?

In [69]:
# Create new column 'price difference' ==> the absolute value of (Adelaide - Melbourne)

adl_mel_df.insert(6, 'Price Difference', abs(adl_mel_df['ADL Price'] - adl_mel_df['MEL Price']))

# Create new column 'Specials Comp' which compares if '$pecial' tag are matching across two stores

adl_mel_df.insert(11, 'Same Specials', adl_mel_df['ADL Specials'] == adl_mel_df['MEL Specials'])

In [103]:
# Let's have a peek at top 5 price differences

adl_mel_df.sort_values('Price Difference', ascending=False).head(20)

,SKU,Brand,Product Name,Category,ADL Price,MEL Price,Price Difference,ADL packaging,MEL packaging,ADL Specials,MEL Specials,Same Specials,ADL Suppliers,MEL Suppliers
10725,4298877P,Coopers,Mild Ale Bottle 375mL,Liquor,44.00,16.0,28.00,6 Pack,6 Pack,,,True,"['800393 COOPERS BREWERY', '79413 COOPERS BREWERY LTD', '804668 PREMIUM BEVERAGES P/L', '963979 MAX LIQUOR SUPPLIES']","['800393 COOPERS BREWERY', '79413 COOPERS BREWERY LTD', '804668 PREMIUM BEVERAGES P/L', '963979 MAX LIQUOR SUPPLIES']"
6906,479615P,Ruski,Lemon Bottle 275mL,Liquor,107.00,91.0,16.00,24 Pack,24 Pack,,,True,"['805291 LIQUOR CONNECTION', '47651 DIAGEO AUST LTD', '804247 VOK BEVERAGES P/L', '12947 G.H.P.L.', '963979 MAX LIQUOR SUPPLIES', '957839 (BS) VOK BEVERAGES P/L']","['805291 LIQUOR CONNECTION', '47651 DIAGEO AUST LTD', '804247 VOK BEVERAGES P/L', '12947 G.H.P.L.', '963979 MAX LIQUOR SUPPLIES', '957839 (BS) VOK BEVERAGES P/L']"
12350,10078P,Victoria Bitter,Bottle 375mL,Liquor,58.00,47.0,11.00,24 Pack,24 Pack,,$pecials,False,['9393 CUB P/L'],['9393 CUB P/L']
11896,1274330P,Canadian Club,& Dry Bottle 500mL,Liquor,90.00,79.0,11.00,12 Pack,12 Pack,,,True,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']
11562,10016P,Victoria Bitter,Can 375mL,Liquor,58.00,48.0,10.00,24 Pack,24 Pack,,$pecials,False,['9393 CUB P/L'],['9393 CUB P/L']
388,2123668P,Coles,Roasted & Salted Cashews,Fruit & Vegetables,10.00,16.0,6.00,800g,800g,,,True,"['97446 CARTER & SPENCER GROUP P/L', '939848 SCALZO FOOD IND', '939312 FRESH PRODUCE DC (NSW)', '942460 FRESH DIRECT BROKERS (IMPORTS)', '50032 WARD MCKENZIE P/L', '414760 ST GEORGE PROD PTY LTD',...","['97446 CARTER & SPENCER GROUP P/L', '939848 SCALZO FOOD IND', '939312 FRESH PRODUCE DC (NSW)', '942460 FRESH DIRECT BROKERS (IMPORTS)', '50032 WARD MCKENZIE P/L', '414760 ST GEORGE PROD PTY LTD',..."
3455,1263016P,Somersby,Pear Cider Can 375mL,Liquor,72.00,66.0,6.00,30 Pack,30 Pack,,,True,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']
8254,539470P,Smirnoff,Ice Red Bottles 300mL,Liquor,105.00,100.0,5.00,24 Pack,24 Pack,,,True,"['805291 LIQUOR CONNECTION', '47651 DIAGEO AUST LTD', '963979 MAX LIQUOR SUPPLIES']","['805291 LIQUOR CONNECTION', '47651 DIAGEO AUST LTD', '963979 MAX LIQUOR SUPPLIES']"
8235,3207774P,Coopers,Pacific Ale Can 375mL,Liquor,58.00,63.0,5.00,24 Pack,24 Pack,,,True,"['79413 COOPERS BREWERY LTD', '963979 MAX LIQUOR SUPPLIES']","['79413 COOPERS BREWERY LTD', '963979 MAX LIQUOR SUPPLIES']"
1175,2821802P,Captain Morgan,Cola Bottle 4.5% 330mL,Liquor,116.00,111.0,5.00,24 pack,24 pack,,,True,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']


#### What products have the same price in both Adelaide and Melbourne?

In [102]:
# ADL vs MEL same price'

same_price_df = adl_mel_df.loc[adl_mel_df['Price Difference'] == 0]

same_price_df.head(20)

,SKU,Brand,Product Name,Category,ADL Price,MEL Price,Price Difference,ADL packaging,MEL packaging,ADL Specials,MEL Specials,Same Specials,ADL Suppliers,MEL Suppliers
0,3468031P,Coles,Natures Kitchen Lentil Spag Bolognese,"Dairy, Eggs & Meals",7.00,7.00,0.0,350g,350g,$pecials,$pecials,True,"['958543 JEWEL OF INDIA ENTERPRISES P/L (', '979047 CHEF FRESH PTY LTD (JFF) (CB)']","['958543 JEWEL OF INDIA ENTERPRISES P/L (', '979047 CHEF FRESH PTY LTD (JFF) (CB)']"
1,2649234P,Guylian,Assortment Belgian Chocolates,Pantry,18.00,18.00,0.0,305g,305g,,,True,['945407 STUART ALEXANDER (CONFECTIONERY)'],['945407 STUART ALEXANDER (CONFECTIONERY)']
2,8416240P,Heinz,Big Red Tomato Soup Can,Pantry,3.65,3.65,0.0,820g,820g,,,True,['22161 HJ HEINZ CO AUST LTD'],['22161 HJ HEINZ CO AUST LTD']
4,324217P,Coles,Smart Buy Hair Spray,Health & Beauty,2.20,2.20,0.0,250g,250g,,,True,"['655091 AARON LABO', '632775 MULTI-FILL P/L HOUSEBRAND']","['655091 AARON LABO', '632775 MULTI-FILL P/L HOUSEBRAND']"
5,5143435P,Nivea,Moisturising Day Face Cream Fluid Spf15,Health & Beauty,5.35,5.35,0.0,50mL,50mL,$pecials,$pecials,True,['936507 BEIERSDORF AUST LTD'],['936507 BEIERSDORF AUST LTD']
6,3716508P,Peters,Connoisseur Strawberry Ice Cream,Frozen,11.00,11.00,0.0,1L,1L,,,True,['968585 PETERS ICE CREAM'],['968585 PETERS ICE CREAM']
7,5430233P,Earth Choice,Pure Soft Fabric Softener,Household,2.00,2.00,0.0,1L,1L,,,True,['653217 NATURES ORGANICS P/L'],['653217 NATURES ORGANICS P/L']
8,3116216P,Cook & Dine,Corkscrew,Household,3.00,3.00,0.0,1 each,1 each,,,True,"['945707 GLOBALEYES DISTRIBUTION P/L (XDO', '970149 Globaleyes Distribution (COLESBR', '975760 GLOBALEYES DISTRIBUTION PTY LTD']","['945707 GLOBALEYES DISTRIBUTION P/L (XDO', '970149 Globaleyes Distribution (COLESBR', '975760 GLOBALEYES DISTRIBUTION PTY LTD']"
9,3671057P,Grillman,BBQ Aluminium Large Roasting Trays,Household,4.80,4.80,0.0,2 pack,2 pack,$pecials,$pecials,True,['972884 MAYO HARDWARE PTY LTD'],['972884 MAYO HARDWARE PTY LTD']
10,3815930P,Tuckers,Natural Artisan Crackers Seeded,Pantry,3.20,3.20,0.0,100g,100g,$pecials,$pecials,True,['933199 FOOD SERV SOLUTIONS P/L'],['933199 FOOD SERV SOLUTIONS P/L']


#### Category breakdown of products with identical prices

In [72]:
print(f"Total number of common products between ADL and MEL: {len(same_price_df)} \n")

print(f"{same_price_df.value_counts('Category')}")

Total number of common products between ADL and MEL: 11811 

Category
Pantry                 3985
Health & Beauty        1674
Back to School         1002
Household               966
Summer Entertaining     721
Dairy, Eggs & Meals     692
Liquor                  612
Frozen                  562
Baby                    345
Pet                     310
International Foods     256
Drinks                  235
Bakery                  146
Meat & Seafood          121
Fruit & Vegetables       98
                         53
From The Deli            18
Convenience Meals        10
New                       4
Clothing                  1
dtype: int64


#### What are the products that are not 'on special' at the same time but have matching prices

In [73]:
same_price_df.loc[same_price_df['Same Specials'] == False]

,SKU,Brand,Product Name,Category,ADL Price,MEL Price,Price Difference,ADL packaging,MEL packaging,ADL Specials,MEL Specials,Same Specials,ADL Suppliers,MEL Suppliers
2286,2860768P,Fairy,Original All In One Lemon Dishwasher Tablets,Household,19.0,19.0,0.0,44 pack,44 pack,,$pecials,False,['422720 PROCTER & GAMBLE AUST P/L'],['422720 PROCTER & GAMBLE AUST P/L']
2552,2289015P,Heinz,Soup Of The Day 7 Veg With Garden Herbs Soup Pouch,Pantry,3.0,3.0,0.0,430g,430g,,$pecials,False,['22161 HJ HEINZ CO AUST LTD'],['22161 HJ HEINZ CO AUST LTD']
3259,2374191P,Great Northern,Super Crisp Lager Bottle 330mL,Liquor,18.0,18.0,0.0,6 Pack,6 Pack,$pecials,,False,['9393 CUB P/L'],['9393 CUB P/L']
4962,3780369P,Vegie Delights,Bolognese Sauce,Pantry,5.0,5.0,0.0,300g,300g,$pecials,,False,['971159 LIFE HEALTH FOODS'],['971159 LIFE HEALTH FOODS']
5110,1163725P,Heinz,Soup of The Day Old Fashioned Chicken Pouch,Pantry,3.0,3.0,0.0,430g,430g,,$pecials,False,['22161 HJ HEINZ CO AUST LTD'],['22161 HJ HEINZ CO AUST LTD']
5423,1163689P,Heinz,Soup Of The Day Buttercup Pumpkin & Veg,Pantry,3.0,3.0,0.0,430g,430g,,$pecials,False,['22161 HJ HEINZ CO AUST LTD'],['22161 HJ HEINZ CO AUST LTD']
7674,2752255P,Coles,Nature's Kitchen Firm Tofu,"Dairy, Eggs & Meals",2.3,2.3,0.0,300g,300g,$pecials,,False,['972547 AUSTRALIAN EATWELL PTY LTD (CB)'],['972547 AUSTRALIAN EATWELL PTY LTD (CB)']
9756,3663060P,Pirate Life,Pale Ale 5.4% Can 355mL,Liquor,59.0,59.0,0.0,16 Pack,16 Pack,$pecials,,False,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']
10000,2244527P,Coles,Ultra Complete Clean Dishwasher 30 Tablets,Household,4.5,4.5,0.0,540g,540g,$pecials,,False,"['971094 MCBRIDE AUSTRALIA PTY LTD (COLES', '145823 OVERSEAS SUPPLIER']","['971094 MCBRIDE AUSTRALIA PTY LTD (COLES', '145823 OVERSEAS SUPPLIER']"


#### Summary of products with same prices

In [93]:
print(f'Total number of products common to both Cole\'s Online in Adelaide and Melbourne stores: {len(adl_mel_df)}\n')
print(f'There are {len(same_price_df)} identical items with identical prices\n')
print(f'There are {len(same_price_df.loc[same_price_df["Same Specials"] == False])} items which have different \"on specials\" status but have the same price\n')
print(f'{round(100*len(same_price_df)/len(adl_mel_df),2)}% of the products common between Adelaide and Melbourne have identical prices\n')

Total number of products common to both Cole's Online in Adelaide and Melbourne stores: 12523

There are 11811 identical items with identical prices

There are 9 items which have different "on specials" status but have the same price

94.31% of the products common between Adelaide and Melbourne have identical prices



#### What are the products with different prices?

In [104]:
diff_price_df = adl_mel_df.loc[adl_mel_df['Price Difference'] != 0]

diff_price_df.sort_values('Price Difference', ascending=False).head(20)

,SKU,Brand,Product Name,Category,ADL Price,MEL Price,Price Difference,ADL packaging,MEL packaging,ADL Specials,MEL Specials,Same Specials,ADL Suppliers,MEL Suppliers
10725,4298877P,Coopers,Mild Ale Bottle 375mL,Liquor,44.00,16.0,28.00,6 Pack,6 Pack,,,True,"['800393 COOPERS BREWERY', '79413 COOPERS BREWERY LTD', '804668 PREMIUM BEVERAGES P/L', '963979 MAX LIQUOR SUPPLIES']","['800393 COOPERS BREWERY', '79413 COOPERS BREWERY LTD', '804668 PREMIUM BEVERAGES P/L', '963979 MAX LIQUOR SUPPLIES']"
6906,479615P,Ruski,Lemon Bottle 275mL,Liquor,107.00,91.0,16.00,24 Pack,24 Pack,,,True,"['805291 LIQUOR CONNECTION', '47651 DIAGEO AUST LTD', '804247 VOK BEVERAGES P/L', '12947 G.H.P.L.', '963979 MAX LIQUOR SUPPLIES', '957839 (BS) VOK BEVERAGES P/L']","['805291 LIQUOR CONNECTION', '47651 DIAGEO AUST LTD', '804247 VOK BEVERAGES P/L', '12947 G.H.P.L.', '963979 MAX LIQUOR SUPPLIES', '957839 (BS) VOK BEVERAGES P/L']"
11896,1274330P,Canadian Club,& Dry Bottle 500mL,Liquor,90.00,79.0,11.00,12 Pack,12 Pack,,,True,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']
12350,10078P,Victoria Bitter,Bottle 375mL,Liquor,58.00,47.0,11.00,24 Pack,24 Pack,,$pecials,False,['9393 CUB P/L'],['9393 CUB P/L']
11562,10016P,Victoria Bitter,Can 375mL,Liquor,58.00,48.0,10.00,24 Pack,24 Pack,,$pecials,False,['9393 CUB P/L'],['9393 CUB P/L']
388,2123668P,Coles,Roasted & Salted Cashews,Fruit & Vegetables,10.00,16.0,6.00,800g,800g,,,True,"['97446 CARTER & SPENCER GROUP P/L', '939848 SCALZO FOOD IND', '939312 FRESH PRODUCE DC (NSW)', '942460 FRESH DIRECT BROKERS (IMPORTS)', '50032 WARD MCKENZIE P/L', '414760 ST GEORGE PROD PTY LTD',...","['97446 CARTER & SPENCER GROUP P/L', '939848 SCALZO FOOD IND', '939312 FRESH PRODUCE DC (NSW)', '942460 FRESH DIRECT BROKERS (IMPORTS)', '50032 WARD MCKENZIE P/L', '414760 ST GEORGE PROD PTY LTD',..."
3455,1263016P,Somersby,Pear Cider Can 375mL,Liquor,72.00,66.0,6.00,30 Pack,30 Pack,,,True,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']
8235,3207774P,Coopers,Pacific Ale Can 375mL,Liquor,58.00,63.0,5.00,24 Pack,24 Pack,,,True,"['79413 COOPERS BREWERY LTD', '963979 MAX LIQUOR SUPPLIES']","['79413 COOPERS BREWERY LTD', '963979 MAX LIQUOR SUPPLIES']"
3963,3479333P,4 Pines Brewing,Pacific Ale Bottle 330mL,Liquor,61.00,56.0,5.00,24 Pack,24 Pack,$pecials,$pecials,True,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']
1046,2554003P,Hammer N Tongs,Bitter Block Can 330mL,Liquor,44.00,39.0,5.00,30 Pack,30 Pack,,,True,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']


In [76]:
# Output to CSV

diff_price_df.to_csv('Outputs/Different Priced Products ADL-MEL 26Jan21.csv', index=False, encoding='utf-8-sig')

In [77]:
print(f'There are {len(diff_price_df)} items with different prices')

There are 712 items with different prices


#### After filtering out products which are not 'on special' simultaneously

In [78]:
# Dataframe after accounting for 'on special' status differences

diff_price_spec_adj_df = diff_price_df.loc[diff_price_df['Same Specials'] == True]

In [79]:
print(f'After adjusting for differences in the "on special" status, there are {len(diff_price_spec_adj_df)} items')

After adjusting for differences in the "on special" status, there are 690 items


In [80]:
print(diff_price_spec_adj_df['Category'].value_counts(normalize=True)*100)

print(f"The top 3 Categories account for {100 * sum(diff_price_spec_adj_df['Category'].value_counts(normalize=True)[:3])}%") 

Liquor                 25.362319
Drinks                 23.768116
Summer Entertaining    22.608696
Pantry                  7.971014
Back to School          7.246377
Fruit & Vegetables      4.492754
Health & Beauty         2.608696
Household               2.028986
Dairy, Eggs & Meals     1.594203
International Foods     0.724638
                        0.434783
Baby                    0.434783
Pet                     0.434783
Frozen                  0.289855
Name: Category, dtype: float64
The top 3 Categories account for 71.73913043478262%


#### Brands most prevelant in products with price differences

In [81]:
print(diff_price_spec_adj_df['Brand'].value_counts(normalize=True).loc[lambda x: x>0.01]*100)

print(f"{len(diff_price_spec_adj_df['Brand'].value_counts(normalize=True).loc[lambda x: x>0.01])} Brands that account for more than 1% of the total, each.")

print(f"These 20 brands account for {sum(diff_price_spec_adj_df['Brand'].value_counts(normalize=True).loc[lambda x: x>0.01]) * 100}% of the list total")

Coles             15.652174
Schweppes          5.942029
Coca-Cola          3.913043
Bundaberg          2.463768
Pepsi              2.318841
Mount Franklin     2.173913
Lipton             1.739130
Red Bull           1.739130
Jack Daniels       1.594203
Kirks              1.449275
MasterFoods        1.304348
Smirnoff           1.304348
Coopers            1.304348
Somersby           1.159420
Powerade           1.159420
Nexba              1.159420
Vodka              1.159420
Madura             1.014493
Monster            1.014493
Pure Blonde        1.014493
Name: Brand, dtype: float64
20 Brands that account for more than 1% of the total, each.
These 20 brands account for 50.57971014492754% of the list total


#### Most common brands with price differences, by category

In [82]:
diff_price_spec_adj_df.groupby('Category')['Brand'].value_counts().loc[lambda x : x>4]

Category             Brand         
Back to School       Coles             26
                     Mount Franklin     5
Drinks               Coles             16
                     Red Bull          12
                     Coca-Cola         11
                     Schweppes         11
                     Mount Franklin    10
                     Bundaberg          8
                     Lipton             8
                     Powerade           8
                     Monster            7
                     Nexba              5
                     Pepsi              5
                     V                  5
Fruit & Vegetables   Coles             31
Liquor               Jack Daniels      10
                     Coopers            8
                     Smirnoff           7
                     Jim Beam           6
                     Pure Blonde        6
                     Somersby           6
                     Captain Morgan     5
                     Woodstock          

#### Inferences Drawn



In [105]:
# What if we exclude all the alcoholic and non-alcoholic beverages?

filter = (
    diff_price_spec_adj_df['Category'].str.contains('Liquor|Drinks') |
    diff_price_spec_adj_df['Product Name'].str.contains('mL|Drink|Water|Cola|Beer|Soda') |
    diff_price_spec_adj_df['Brand'].isin(['Schweppes', 'Coca-Cola', 'Pepsi', 'Mountain Dew',
                                          'Lipton', 'Mount Franklin', 'Fanta', 'Sunkist'])
)

nonbev_diff_price_df = diff_price_spec_adj_df.loc[~filter]
nonbev_diff_price_df.sort_values('Price Difference', ascending=False).head(20)

,SKU,Brand,Product Name,Category,ADL Price,MEL Price,Price Difference,ADL packaging,MEL packaging,ADL Specials,MEL Specials,Same Specials,ADL Suppliers,MEL Suppliers
388,2123668P,Coles,Roasted & Salted Cashews,Fruit & Vegetables,10.00,16.00,6.00,800g,800g,,,True,"['97446 CARTER & SPENCER GROUP P/L', '939848 SCALZO FOOD IND', '939312 FRESH PRODUCE DC (NSW)', '942460 FRESH DIRECT BROKERS (IMPORTS)', '50032 WARD MCKENZIE P/L', '414760 ST GEORGE PROD PTY LTD',...","['97446 CARTER & SPENCER GROUP P/L', '939848 SCALZO FOOD IND', '939312 FRESH PRODUCE DC (NSW)', '942460 FRESH DIRECT BROKERS (IMPORTS)', '50032 WARD MCKENZIE P/L', '414760 ST GEORGE PROD PTY LTD',..."
4364,2123602P,Coles,Natural Almonds,Fruit & Vegetables,19.00,16.00,3.00,800g,800g,,,True,"['933347 COSTAS (TASMANIA)', '939848 SCALZO FOOD IND', '939312 FRESH PRODUCE DC (NSW)', '97446 CARTER & SPENCER GROUP P/L', '942633 CARTER & SPENCER NORTH QLD', '414760 ST GEORGE PROD PTY LTD', ""6...","['933347 COSTAS (TASMANIA)', '939848 SCALZO FOOD IND', '939312 FRESH PRODUCE DC (NSW)', '97446 CARTER & SPENCER GROUP P/L', '942633 CARTER & SPENCER NORTH QLD', '414760 ST GEORGE PROD PTY LTD', ""6..."
8711,2743866P,Coles,Kestrel Potatoes Prepacked,Fruit & Vegetables,7.00,4.00,3.00,2kg,2kg,,,True,"[""635235 COSTA'S P/L (VIC DC)"", '414760 ST GEORGE PROD PTY LTD', '942633 CARTER & SPENCER NORTH QLD', '97446 CARTER & SPENCER GROUP P/L', '939301 FRESH PRODUCE DC (WA)', '939312 FRESH PRODUCE DC (...","[""635235 COSTA'S P/L (VIC DC)"", '414760 ST GEORGE PROD PTY LTD', '942633 CARTER & SPENCER NORTH QLD', '97446 CARTER & SPENCER GROUP P/L', '939301 FRESH PRODUCE DC (WA)', '939312 FRESH PRODUCE DC (..."
490,4879042P,Coles,Wombok Chinese Cabbage Whole,Fruit & Vegetables,3.00,5.90,2.90,1 each,1 each,,,True,"['939301 FRESH PROD', '414760 ST GEORGE', '939312 FRESH PROD', '97446 CARTER & S', '942633 CARTER & S', ""635235 COSTA'S P/L (VIC DC)"", '933347 COSTAS (TA']","['939301 FRESH PROD', '414760 ST GEORGE', '939312 FRESH PROD', '97446 CARTER & S', '942633 CARTER & S', ""635235 COSTA'S P/L (VIC DC)"", '933347 COSTAS (TA']"
5140,3866840P,Coles,Garlic Bulb Prepack,Fruit & Vegetables,0.02,2.50,2.48,4 pack,4 pack,,,True,"['414760 ST GEORGE PROD PTY LTD', ""635235 COSTA'S P/L (VIC DC)"", '939301 FRESH PRODUCE DC (WA)', '942633 CARTER & SPENCER NORTH QLD', '97446 CARTER & SPENCER GROUP P/L', '933347 COSTAS (TASMANIA)'...","['414760 ST GEORGE PROD PTY LTD', ""635235 COSTA'S P/L (VIC DC)"", '939301 FRESH PRODUCE DC (WA)', '942633 CARTER & SPENCER NORTH QLD', '97446 CARTER & SPENCER GROUP P/L', '933347 COSTAS (TASMANIA)'..."
9531,4845732P,Coles,Celery,Summer Entertaining,3.50,2.00,1.50,1 bunch,1 bunch,,,True,"['933347 COSTAS (TASMANIA)', '939301 FRESH PRODUCE DC (WA)', ""635235 COSTA'S P/L (VIC DC)"", '97446 CARTER & SPENCER GROUP P/L', '939312 FRESH PRODUCE DC (NSW)', '942633 CARTER & SPENCER NORTH QLD'...","['933347 COSTAS (TASMANIA)', '939301 FRESH PRODUCE DC (WA)', ""635235 COSTA'S P/L (VIC DC)"", '97446 CARTER & SPENCER GROUP P/L', '939312 FRESH PRODUCE DC (NSW)', '942633 CARTER & SPENCER NORTH QLD'..."
2731,9932765P,Lifestyles,Zero Uber Thin Condoms,Health & Beauty,15.00,16.50,1.50,20 pack,20 pack,,,True,"['974263 SXWELL AUSTRALIA PTY LTD', '850 ANSELL HEALTHCARE']","['974263 SXWELL AUSTRALIA PTY LTD', '850 ANSELL HEALTHCARE']"
9407,4456832P,Coles,Wombok Cabbage Half,Fruit & Vegetables,2.00,3.50,1.50,1 each,1 each,,,True,"['939312 FRESH PRODUCE DC (NSW)', '942633 CARTER & SPENCER NORTH QLD', '939301 FRESH PRODUCE DC (WA)', '97446 CARTER & SPENCER GROUP P/L', '414760 ST GEORGE PROD PTY LTD', '933347 COSTAS (TASMANIA...","['939312 FRESH PRODUCE DC (NSW)', '942633 CARTER & SPENCER NORTH QLD', '939301 FRESH PRODUCE DC (WA)', '97446 CARTER & SPENCER GROUP P/L', '414760 ST GEORGE PROD PTY LTD', '933347 COSTAS (TASMANIA..."
12442,8900989P,Aptamil,Gold+ 2 Baby Follow-On Formula From 6-12 Months,Baby,23.60,24.80,1.20,900g,900g,,,True,['665274 NUTRICIA AUST P/L'],['665274 NUTRICIA AUST P/L']
6366,6763800P,Apta

In [84]:
print(f'Total list of items: {len(nonbev_diff_price_df)}')

Total list of items: 183


In [85]:
nonbev_diff_price_df.groupby('Brand')['Category'].value_counts().loc[lambda x: x>1]

Brand          Category           
Aptamil        Baby                    2
Coles          Fruit & Vegetables     30
               Summer Entertaining    16
               Back to School         15
               Pantry                  3
Colgate        Health & Beauty         2
Continental    Pantry                  6
Dettol         Health & Beauty         2
Earth Choice   Household               3
Kan Tong Thai  International Foods     2
Kleenex        Household               3
Listerine      Health & Beauty         3
Madura         Pantry                  7
MasterFoods    Pantry                  9
McKenzie's     Pantry                  2
Nerada         Pantry                  2
Nexba          Summer Entertaining     2
Nivea          Health & Beauty         2
Orgran         Pantry                  3
Swisse         Health & Beauty         2
Total Care     Pet                     3
Name: Category, dtype: int64

#### Summary Insights and Inferences

There are 712 items with different prices. After adjusting for differences in the "on special" status, there are 690 items.

The top 3 Categories account for 71.74% of the total items list:
* Liquor                 25.4%
* Drinks                 23.77%
* Summer Entertaining    22.61%

20 Brands (each of which account for more than 1% of the total) accounted for 50.58% of the list of items with different prices between Adelaide and Melbourne. The largest by far was Coles private label, at 15.65%.

Coles private label accounts for a significant proportion of products that are priced differently between Adelaide and Melbourne. This implies that Coles passes on all costs to the shoppers and their margins on these products are most likely uniform throughout Australia. This also implies that, potentially, some shoppers in certain regions of Australia are _de facto_ subsidising other shoppers when they purchase products with identical prices across different geographical locations in Australia. These products are likely not the most competitively priced in the marketplace and may be a potential area for competitor supermarkets to focus on exploiting. Another possibility is that Coles forces the suppliers to bear the "subsidies" for remote areas i.e. making a loss on the products in remote areas while making up the losses with larger profitable sales volume in metropolitan areas.

It seems most of the price difference are in the Liquor and Drinks category. The Summer Entertaining products are predominantly alcoholic and non-alcoholic drinks - Cole's API only has one label for these products, even though they can be found in the Liquor or Drinks categories on their website.

Almost all other brands are all part of the portfolio of large global food and beverage companies - companies with market caps much bigger than Coles. This may explain why they are also able to price their products differently in different regions. I wouldn't be surprised if they are the ones dictating the terms to Coles.

After filtering out products related to big beverage brands, random Colas or Sodas, and different bottles of water (Coles private label or otherwise), the biggest brand by far in the remaining list of items was Coles private brand - accounting for about 35% of that group.

#### Investigation of some oddities + commentaries

It looks like the vast majority of the products are alcohol or non-alcoholic beverages. Some of largest price differences are shocking to say the least... 

**Crazy Price Difference in a 6-pack of Coopers**

<img src="https://i.postimg.cc/YCR2xBvP/Coopers-WTF.png" />

However, I suspected that the product with the largest difference (almost 3x difference in price!!) may have been a data integrity issue. 

Afterall, it was Australia Day and someone may have knocked back a few too many too early!

I looked up Coopers Mild Ale Bottle 375ml 6-pack a few days later for both locations. My suspicions were indeed proven correct - see screenshots below (taken at the start of February):

**Adelaide**

<img src="https://i.postimg.cc/BnSrbk4m/Coopers-Mild-Ale-ADL.png" title="Adelaide Coopers Mild Ale Price"
     width="1000" height="700" />

**Melbourne**

<img src="https://i.postimg.cc/L5vVbyws/Coopers-Mild-Ale-MEL.png" title="Melbourne Coopers Mild Ale Price"
     width = "1000" height = "700" />


<br></br>

What's interesting here is the fact that the \$44 price tag is almost certainly a Liquorland price tag for a 24-pack of Coopers Mild Ale 375mL Bottle. I only had data from early January of Liquorland prices for the state of Victoria and after pulling up the data, I am almost certain that these prices are ported over from Liquorland. Does this mistake imply that some of these processes are still manual between Liquorland and Coles? Surely, they should've integrated their backend by now??

**Liquorland Coopers Mild Ale 24-pack price VIC**

<img src="https://i.postimg.cc/9MbZZnGt/liquorland-coopers-mild-ale-prices.png" />
<br></br>

As for why the vast majority of price differences are with bottle/canned beverages... I can only make some speculations. One thing that came to mind was the 10c refund label that you always find on bottles and cans. I never really looked into it but a quick look on [Wikipedia][] confirms that South Australia indeed _"charges a refundable deposit of AUD 10 cents per drinks carton, can or bottle (only containers marked as eligible for refund; does not include wine & spirits bottles, milk cartons, or concentrated/and or vegetable juice intended to be diluted before consumption)"._ It turns out this has been in place since 1977 and Victoria is expected to adopt a similar scheme by 2023, according to Wikipedia.

[Wikipedia]: https://en.wikipedia.org/wiki/Container-deposit_legislation#Australia "Wikipedia"

However, this explanation falls short in explaining all of the price differences e.g. Crown lager 24-pack would be expected to be \\$2.40 more expensive in Adelaide than in Melbourne, *ceteris paribus*, however, the actual difference is \\$4. Other products have a less than expected amount of difference e.g. Melbourne Bitters 375ml Can 6-pack only has a \\$1 difference. 

This leads me to my next questions...

I wonder if Adelaide has any Liquor/Drinks products which are cheaper than Melbourne?

If the answer is 'yes', then that would imply that Coles passes on logistics costs to the end consumers (at least in the Liquor/Drinks category).


In [106]:
"""
Define filter: 
    Price difference is not zero, 
    Price difference of the Adelaide product subtracted from the Melbourne product is positive,
    The '$pecials' tag match
"""

filter = (
    (adl_mel_df['Price Difference'] != 0) &
    (adl_mel_df['MEL Price'] - adl_mel_df['ADL Price'] > 0) &
    (adl_mel_df['Same Specials'] == True)
)

adl_mel_df.loc[filter].sort_values('Price Difference', ascending=False).head(20)


,SKU,Brand,Product Name,Category,ADL Price,MEL Price,Price Difference,ADL packaging,MEL packaging,ADL Specials,MEL Specials,Same Specials,ADL Suppliers,MEL Suppliers
388,2123668P,Coles,Roasted & Salted Cashews,Fruit & Vegetables,10.00,16.00,6.00,800g,800g,,,True,"['97446 CARTER & SPENCER GROUP P/L', '939848 SCALZO FOOD IND', '939312 FRESH PRODUCE DC (NSW)', '942460 FRESH DIRECT BROKERS (IMPORTS)', '50032 WARD MCKENZIE P/L', '414760 ST GEORGE PROD PTY LTD',...","['97446 CARTER & SPENCER GROUP P/L', '939848 SCALZO FOOD IND', '939312 FRESH PRODUCE DC (NSW)', '942460 FRESH DIRECT BROKERS (IMPORTS)', '50032 WARD MCKENZIE P/L', '414760 ST GEORGE PROD PTY LTD',..."
8235,3207774P,Coopers,Pacific Ale Can 375mL,Liquor,58.00,63.00,5.00,24 Pack,24 Pack,,,True,"['79413 COOPERS BREWERY LTD', '963979 MAX LIQUOR SUPPLIES']","['79413 COOPERS BREWERY LTD', '963979 MAX LIQUOR SUPPLIES']"
490,4879042P,Coles,Wombok Chinese Cabbage Whole,Fruit & Vegetables,3.00,5.90,2.90,1 each,1 each,,,True,"['939301 FRESH PROD', '414760 ST GEORGE', '939312 FRESH PROD', '97446 CARTER & S', '942633 CARTER & S', ""635235 COSTA'S P/L (VIC DC)"", '933347 COSTAS (TA']","['939301 FRESH PROD', '414760 ST GEORGE', '939312 FRESH PROD', '97446 CARTER & S', '942633 CARTER & S', ""635235 COSTA'S P/L (VIC DC)"", '933347 COSTAS (TA']"
5140,3866840P,Coles,Garlic Bulb Prepack,Fruit & Vegetables,0.02,2.50,2.48,4 pack,4 pack,,,True,"['414760 ST GEORGE PROD PTY LTD', ""635235 COSTA'S P/L (VIC DC)"", '939301 FRESH PRODUCE DC (WA)', '942633 CARTER & SPENCER NORTH QLD', '97446 CARTER & SPENCER GROUP P/L', '933347 COSTAS (TASMANIA)'...","['414760 ST GEORGE PROD PTY LTD', ""635235 COSTA'S P/L (VIC DC)"", '939301 FRESH PRODUCE DC (WA)', '942633 CARTER & SPENCER NORTH QLD', '97446 CARTER & SPENCER GROUP P/L', '933347 COSTAS (TASMANIA)'..."
5319,4205264P,Coles,Watermelon Seedless Whole,Fruit & Vegetables,9.60,12.00,2.40,approx. 8kg each,approx. 8kg each,,,True,"['97446 CARTER & S', '939312 FRESH PROD', '414760 ST GEORGE', '933347 COSTAS (TA', '939301 FRESH PROD', '942633 CARTER & S', ""635235 COSTA'S P/L (VIC DC)"", '656186 COLES SUPERMARKETS (EXP)']","['97446 CARTER & S', '939312 FRESH PROD', '414760 ST GEORGE', '933347 COSTAS (TA', '939301 FRESH PROD', '942633 CARTER & S', ""635235 COSTA'S P/L (VIC DC)"", '656186 COLES SUPERMARKETS (EXP)']"
9407,4456832P,Coles,Wombok Cabbage Half,Fruit & Vegetables,2.00,3.50,1.50,1 each,1 each,,,True,"['939312 FRESH PRODUCE DC (NSW)', '942633 CARTER & SPENCER NORTH QLD', '939301 FRESH PRODUCE DC (WA)', '97446 CARTER & SPENCER GROUP P/L', '414760 ST GEORGE PROD PTY LTD', '933347 COSTAS (TASMANIA...","['939312 FRESH PRODUCE DC (NSW)', '942633 CARTER & SPENCER NORTH QLD', '939301 FRESH PRODUCE DC (WA)', '97446 CARTER & SPENCER GROUP P/L', '414760 ST GEORGE PROD PTY LTD', '933347 COSTAS (TASMANIA..."
3944,2580466P,Jack Daniels,& No Sugar Cola Can 375mL,Liquor,23.50,25.00,1.50,4 Pack,4 Pack,,,True,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']
2731,9932765P,Lifestyles,Zero Uber Thin Condoms,Health & Beauty,15.00,16.50,1.50,20 pack,20 pack,,,True,"['974263 SXWELL AUSTRALIA PTY LTD', '850 ANSELL HEALTHCARE']","['974263 SXWELL AUSTRALIA PTY LTD', '850 ANSELL HEALTHCARE']"
6366,6763800P,Aptamil,Gold+ 1 Baby Infant Formula From Birth to 6 Months,Baby,23.60,24.80,1.20,900g,900g,,,True,['665274 NUTRICIA AUST P/L'],['665274 NUTRICIA AUST P/L']
12442,8900989P,Aptamil,Gold+ 2 Baby Follow-On Formula From 6-12 Months,Baby,23.60,24.80,1.20,900g,900g,,,True,['665274 NUTRICIA AUST P/L'],['665274 NUTRICIA AUST P/L']


#### Forgetting something... and revealing something interesting...

I had forgotten to ensure that both products being compared have the same 'on special' status and this revealed something pretty interesting...

Why is Coopers Original Pale Ale 24-pack and Coopers Pale Ale 24-pack in Melbourne, ON SPECIAL, more expensive than in Adelaide (NOT on special)?

<img src='https://i.postimg.cc/sxL1jRRx/speshul-indeed.png' width="600" height="400"/>

I manually grabbed screenshots after visiting the corresponding sites a few days later while conducting this analysis to see for myself. The screenshots below confirm that these two products are indeed much more expensive in Melbourne than in Adelaide. 

I looked up Coopers Brewing and it turns out they're based out of Adelaide - which can explain why a slab of Coopers is much cheaper in Adelaide than Melbourne. However, what puzzles me is the fact that a 6-pack of Coopers Mild Ale in 375mL bottles (pictured above) is cheaper in Melbourne...

**Coopers Pale Ale in Adelaide (04 FEB 2021)**
<img src='https://i.postimg.cc/tJzQk3Bj/Coopers-OG-pale-ale-ADL-normal-price.png' width="1000" height="700" />


**Coopers Pale Ale in Melbourne (04 FEB 2021)**
<img src='https://i.postimg.cc/kgG3Nt80/Coopers-OG-pale-ale-MEL-normal-price.png' width="1000" height="700" />

<br>

## \\$pecials

* What percentage of total products are on special and in what categories?
* What products are still comparatively more expensive while on special? 

From the above case, we have already established that "specials" are not uniform between states. Most likely, different products are on special at different Coles stores even within states. 



#### What percentage of products are 'on special'?

In [87]:
# Proportion of products 'on special' in both cities

print(adl_df['ADL Specials'].value_counts(normalize=True)*100)
print('\n')
print(mel_df['MEL Specials'].value_counts(normalize=True)*100)

            70.326994
$pecials    29.673006
Name: ADL Specials, dtype: float64


            69.789715
$pecials    30.210285
Name: MEL Specials, dtype: float64


#### Category breakdown

In [88]:
ald_specials = adl_df.loc[adl_df['ADL Specials'] == '$pecials']
ald_specials['Category'].value_counts(normalize=True)*100

Pantry                  23.716381
Health & Beauty         21.463500
Back to School           9.028991
Summer Entertaining      7.334963
Liquor                   6.933287
Household                6.444289
Dairy, Eggs & Meals      6.356968
Frozen                   5.029689
Pet                      2.951450
Drinks                   2.724415
International Foods      2.322738
Baby                     2.078240
Bakery                   1.414600
Meat & Seafood           1.187565
                         0.419141
Clothing                 0.314356
From The Deli            0.139714
Convenience Meals        0.104785
New                      0.017464
Chilled & Fresh Food     0.017464
Name: Category, dtype: float64

In [89]:
mel_specials = mel_df.loc[mel_df['MEL Specials'] == '$pecials']
mel_specials['Category'].value_counts(normalize=True)*100

Pantry                 28.095566
Health & Beauty        14.334941
Liquor                 11.026878
Back to School          9.441764
Summer Entertaining     8.982311
Dairy, Eggs & Meals     6.432346
Household               5.605330
Frozen                  4.181025
Drinks                  3.422927
Pet                     2.136458
International Foods     2.044567
Baby                    2.044567
Bakery                  1.056742
Meat & Seafood          0.689180
                        0.413508
From The Deli           0.045945
Clothing                0.022973
Convenience Meals       0.022973
Name: Category, dtype: float64

#### What products have mismatched '\\$pecials' tag?

In [96]:
diff_specials_df = adl_mel_df.loc[adl_mel_df['Same Specials'] == False].sort_values('MEL Specials')
diff_specials_df.sort_values('Price Difference', ascending=False)

,SKU,Brand,Product Name,Category,ADL Price,MEL Price,Price Difference,ADL packaging,MEL packaging,ADL Specials,MEL Specials,Same Specials,ADL Suppliers,MEL Suppliers
12350,10078P,Victoria Bitter,Bottle 375mL,Liquor,58.000,47.000,11.000,24 Pack,24 Pack,,$pecials,False,['9393 CUB P/L'],['9393 CUB P/L']
11562,10016P,Victoria Bitter,Can 375mL,Liquor,58.000,48.000,10.000,24 Pack,24 Pack,,$pecials,False,['9393 CUB P/L'],['9393 CUB P/L']
5390,9246021P,Coles,Lamb Leg Roast Whole,Meat & Seafood,27.945,31.590,3.645,approx. 2.43kg,approx. 2.43kg,$pecials,,False,"['934576 AUSTRALIAN LAMB (COLAC) PTY LTD', ""942941 SCHULZ'S WHOLESALE MEATS"", '970621 G&B GATHERCOLE PTY LTD', '921481 COLES MT PROD - C9359MT - JBS BF', '732222 AUSTRALIAN LAMB CO P/L', '343811 W...","['934576 AUSTRALIAN LAMB (COLAC) PTY LTD', ""942941 SCHULZ'S WHOLESALE MEATS"", '970621 G&B GATHERCOLE PTY LTD', '921481 COLES MT PROD - C9359MT - JBS BF', '732222 AUSTRALIAN LAMB CO P/L', '343811 W..."
5025,416504P,Cascade,Premium Light Bottle 375mL,Liquor,45.000,42.000,3.000,24 Pack,24 Pack,$pecials,,False,['9393 CUB P/L'],['9393 CUB P/L']
4567,3330690P,Coopers,Pale Ale Can 375mL,Liquor,51.000,54.000,3.000,24 Pack,24 Pack,,$pecials,False,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']
12269,49665P,Coopers,Original Pale Ale Bottle 375mL,Summer Entertaining,51.000,54.000,3.000,24 Pack,24 Pack,,$pecials,False,"['805291 LIQUOR CONNECTION', '79413 COOPERS BREWERY LTD', '963979 MAX LIQUOR SUPPLIES', '804668 PREMIUM BEVERAGES P/L']","['805291 LIQUOR CONNECTION', '79413 COOPERS BREWERY LTD', '963979 MAX LIQUOR SUPPLIES', '804668 PREMIUM BEVERAGES P/L']"
5705,8598420P,Coles,Butcher Lamb Loin Chops,Meat & Seafood,10.854,8.040,2.814,approx. 402g,approx. 402g,,$pecials,False,"['971765 RROA', '656186 COLES SUPERMARKETS (EXP)', '931767 SOMERVILLE RETAIL SERVICES P/L', '976906 BINDAREE FOOD GROUP (RW) (CB)', '650790 AUSTRALIAN COUNTRY CHOICE', '971010 PRIME LINK']","['971765 RROA', '656186 COLES SUPERMARKETS (EXP)', '931767 SOMERVILLE RETAIL SERVICES P/L', '976906 BINDAREE FOOD GROUP (RW) (CB)', '650790 AUSTRALIAN COUNTRY CHOICE', '971010 PRIME LINK']"
7327,8600952P,Coles,Pork Spare Ribs,Meat & Seafood,18.320,15.572,2.748,approx. 916g Each,approx. 916g Each,,$pecials,False,"['931767 SOMERVILLE RETAIL SERVICES P/L', '656186 COLES SUPERMARKETS (EXP)', '971765 RROA', '650790 AUSTRALIAN COUNTRY CHOICE']","['931767 SOMERVILLE RETAIL SERVICES P/L', '656186 COLES SUPERMARKETS (EXP)', '971765 RROA', '650790 AUSTRALIAN COUNTRY CHOICE']"
3716,311021P,Victoria Bitter,Can 375mL,Liquor,20.000,18.000,2.000,6 Pack,6 Pack,,$pecials,False,['9393 CUB P/L'],['9393 CUB P/L']
7540,311007P,Victoria Bitter,Bottle 375mL,Liquor,20.000,18.000,2.000,6 Pack,6 Pack,,$pecials,False,['9393 CUB P/L'],['9393 CUB P/L']


#### Which products are 'on special' but are still more expensive than the identical product in the other city?

In [99]:
# Speshul pRoDucTs - on special but still more expensive than the other city's product which isn't on special

adl_diff_specials = diff_specials_df[diff_specials_df['ADL Price'] > diff_specials_df['MEL Price']].loc[diff_specials_df['ADL Specials'] == "$pecials"]
mel_diff_specials = diff_specials_df[diff_specials_df['MEL Price'] > diff_specials_df['ADL Price']].loc[diff_specials_df['MEL Specials'] == "$pecials"]

super_special_df = pd.concat([adl_diff_specials, mel_diff_specials])
super_special_df

,SKU,Brand,Product Name,Category,ADL Price,MEL Price,Price Difference,ADL packaging,MEL packaging,ADL Specials,MEL Specials,Same Specials,ADL Suppliers,MEL Suppliers
5025,416504P,Cascade,Premium Light Bottle 375mL,Liquor,45.0,42.0,3.0,24 Pack,24 Pack,$pecials,,False,['9393 CUB P/L'],['9393 CUB P/L']
10719,5129875P,Daily Juice,Apple Juice Chilled,Drinks,4.8,5.0,0.2,2L,2L,,$pecials,False,"['694789 BERRI LTD', '4111 LD & D AUST P/L']","['694789 BERRI LTD', '4111 LD & D AUST P/L']"
12269,49665P,Coopers,Original Pale Ale Bottle 375mL,Summer Entertaining,51.0,54.0,3.0,24 Pack,24 Pack,,$pecials,False,"['805291 LIQUOR CONNECTION', '79413 COOPERS BREWERY LTD', '963979 MAX LIQUOR SUPPLIES', '804668 PREMIUM BEVERAGES P/L']","['805291 LIQUOR CONNECTION', '79413 COOPERS BREWERY LTD', '963979 MAX LIQUOR SUPPLIES', '804668 PREMIUM BEVERAGES P/L']"
4567,3330690P,Coopers,Pale Ale Can 375mL,Liquor,51.0,54.0,3.0,24 Pack,24 Pack,,$pecials,False,['963979 MAX LIQUOR SUPPLIES'],['963979 MAX LIQUOR SUPPLIES']
4330,5738160P,Daily Juice,Pulp Free Orange Juice Chilled,Drinks,4.8,5.0,0.2,2L,2L,,$pecials,False,"['4111 LD & D AUST P/L', '694789 BERRI LTD']","['4111 LD & D AUST P/L', '694789 BERRI LTD']"
2881,4745997P,The Daily Juice,Breakfast Burst Juice Chilled,Drinks,4.8,5.0,0.2,2L,2L,,$pecials,False,"['694789 BERRI LTD', '4111 LD & D AUS']","['694789 BERRI LTD', '4111 LD & D AUS']"
